In [9]:
import pandas as pd
import numpy as np

# Function to classify Köppen climate based on temperature and precipitation
def classify_koeppen(temperature, precipitation, months):
    # Convert to numeric arrays
    temperature = pd.to_numeric(temperature, errors='coerce')
    precipitation = pd.to_numeric(precipitation, errors='coerce')
    
    # Compute necessary variables
    MAT = np.nanmean(temperature)  # Mean Annual Temperature
    MAP = np.nansum(precipitation)  # Mean Annual Precipitation
    Tmin = np.nanmin(temperature)  # Minimum monthly temperature
    Tmax = np.nanmax(temperature)  # Maximum monthly temperature
    
    # Winter months (October to March)
    winter_mask = months.isin([10, 11, 12, 1, 2, 3])
    PWmin = np.nanmin(precipitation[winter_mask]) if winter_mask.any() else np.nan  # Handle empty mask
    PWmax = np.nanmax(precipitation[winter_mask]) if winter_mask.any() else np.nan  # Handle empty mask
    
    # Summer months (April to September)
    summer_mask = months.isin(range(4, 10))
    PSmin = np.nanmin(precipitation[summer_mask]) if summer_mask.any() else np.nan  # Handle empty mask
    PSmax = np.nanmax(precipitation[summer_mask]) if summer_mask.any() else np.nan  # Handle empty mask
    
    # Minimum monthly precipitation
    Pmin = np.nanmin(precipitation)
    
    # Total precipitation for summer and winter
    MAP_summer = np.nansum(precipitation[summer_mask]) if summer_mask.any() else 0
    MAP_winter = np.nansum(precipitation[winter_mask]) if winter_mask.any() else 0
    
    # Calculate Pthreshold based on the criteria
    if MAP_winter >= 0.7 * MAP:
        Pthreshold = 2 * MAT
    elif MAP_summer >= 0.7 * MAP:
        Pthreshold = 2 * MAT + 28
    else:
        Pthreshold = 2 * MAT + 14
    
    # Köppen classification logic
    if Tmax < 10:
        if Tmax > 0:
            return "ET"  # Tundra
        else:
            return "EF"  # Frost
    
    elif MAP < 10 * Pthreshold:
        if MAP < 5 * Pthreshold:
            if MAT >= 18:
                return "BWh"  # Hot Desert
            else:
                return "BWk"  # Cold Desert
        else:
            if MAT >= 18:
                return "BSh"  # Hot Steppe
            else:
                return "BSk"  # Cold Steppe
    
    elif Tmin >= 18:
        if Pmin >= 60:
            return "Af"  # Rainforest
        elif Pmin >= 100 - MAP / 25:
            return "Am"  # Monsoon
        elif PSmin < 100 - MAP / 25:
            return "As"  # Savannah with dry summer
        elif PWmin < 100 - MAP / 25:
            return "Aw"  # Savannah with dry winter
    
    elif Tmax > 10 and 0 < Tmin < 18:
        if PSmin < 40 and PSmin < PWmax / 3:
            if Tmax >= 22:
                return "Csa"  # Hot Summer Dry Summer
            elif Tmax < 22 and np.sum(temperature > 10) >= 4:
                return "Csb"  # Warm Summer Dry Summer
            elif Tmax < 22 and np.sum(temperature > 10) < 4:
                return "Csc"  # Cold Summer Dry Summer
        elif PWmin < PSmax / 10:
            if Tmax >= 22:
                return "Cwa"  # Hot Summer Dry Winter
            elif Tmax < 22 and np.sum(temperature > 10) >= 4:
                return "Cwb"  # Warm Summer Dry Winter
            elif Tmax < 22 and np.sum(temperature > 10) < 4:
                return "Cwc"  # Cold Summer Dry Winter
        else:
            if Tmax >= 22:
                return "Cfa"  # Hot Summer Without dry season
            elif Tmax < 22 and np.sum(temperature > 10) >= 4:
                return "Cfb"  # Warm Summer Without dry season
            elif Tmax < 22 and np.sum(temperature > 10) < 4:
                return "Cfc"  # Cold Summer Without dry season
    
    elif Tmax > 10 and Tmin <= 0:
        if PSmin < 40 and PSmin < PWmax / 3:
            if Tmin < -38:
                return "Dsd"  # Very Cold Winter Dry Summer
            elif Tmax >= 22:
                return "Dsa"  # Hot Summer Dry Summer
            elif Tmax < 22 and np.sum(temperature > 10) >= 4:
                return "Dsb"  # Warm Summer Dry Summer
            elif Tmax < 22 and np.sum(temperature > 10) < 4:
                return "Dsc"  # Cold Summer Dry Summer
        elif PWmin < PSmax / 10:
            if Tmin < -38:
                return "Dwd"  # Very Cold Winter Dry Winter
            elif Tmax >= 22:
                return "Dwa"  # Hot Summer Dry Winter
            elif Tmax < 22 and np.sum(temperature > 10) >= 4:
                return "Dwb"  # Warm Summer Dry Winter
            elif Tmax < 22 and np.sum(temperature > 10) < 4:
                return "Dwc"  # Cold Summer Dry Winter
        else:
            if Tmax >= 22:
                return "Dfa"  # Hot Summer Without dry season
            elif Tmax < 22 and np.sum(temperature > 10) >= 4:
                return "Dfb"  # Warm Summer Without dry season
            elif Tmax < 22 and np.sum(temperature > 10) < 4 and Tmin >= -38:
                return "Dfc"  # Cold Summer Without dry season
            elif Tmin < -38:
                return "Dfd"  # Very Cold Winter
    
    return "Unknown"

# Load the monthly averages data with index_col=0 to set the first column as index
temp_file = r"D:\climate change\monthly_averages.xlsx"
rain_file = r"D:\climate change\monthly_averages.xlsx"  # Assuming rainfall is in the same file

# Read the Excel files with index_col=0
temp_data = pd.read_excel(temp_file, sheet_name=None, index_col=0)
rain_data = pd.read_excel(rain_file, sheet_name=None, index_col=0)

# Use the first sheet for processing (adjust sheet name if needed, e.g., 'T_mean_monthly' or similar)
temp_df = temp_data[list(temp_data.keys())[0]].drop(['Lon', 'Lat'])
rain_df = rain_data[list(rain_data.keys())[0]].drop(['Lon', 'Lat'])

# Ensure the index is a DatetimeIndex
temp_df.index = pd.to_datetime(temp_df.index)
rain_df.index = pd.to_datetime(rain_df.index)

# Extract months from the index
months = temp_df.index.month

# Get data values
temp_data_values = temp_df.values
rain_data_values = rain_df.values

# Get station names
station_names = temp_df.columns

# Initialize a list to store classification results
koeppen_classifications = []

# Loop through each station to classify the climate
for i in range(temp_data_values.shape[1]):
    temperature = temp_data_values[:, i]
    precipitation = rain_data_values[:, i]
    
    classification = classify_koeppen(temperature, precipitation, months)
    koeppen_classifications.append({'Station': station_names[i], 'Classification': classification})

# Convert to DataFrame
koeppen_df = pd.DataFrame(koeppen_classifications)

# Save the classification results to CSV
output_path = r"D:\climate change\koeppen_classifications.csv"
koeppen_df.to_csv(output_path, index=False)

print(f"Köppen classifications saved to {output_path}")

Köppen classifications saved to D:\climate change\koeppen_classifications.csv
